In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [75]:
using StatsBase, Distributions, Plots, LinearAlgebra, Plots

In [3]:
using Revise

includet("../src/ESGModels/msvar.jl")
includet("../src/ESGModels/msvar_simulate.jl") 

In [4]:
Β1 = [.09 .3 .2; .03 .1 .6]
Σ1 = cor2cov([1 .3; .3 1], [.08, .03])
Β2 = [-.02 .35 .25; .035 .15 .63]
Σ2 = cor2cov([1 .35; .35 1], [.18, .035])


transition_matrix = [.85 .15;.2 .8]

states_zero = [.9, .1]
X = [.12, .06]


n_samples = 1000
test_regimes = simulate_regimes(states_zero, transition_matrix, n_samples, 1)
test_returns = simulate_msvar_returns(test_regimes, [Β1, Β2], [Σ1, Σ2], X, n_samples, 1)[:,:,1]'

1000×2 adjoint(::Matrix{Float64}) with eltype Float64:
  0.0594498  0.0876271
  0.0585175  0.0741366
 -0.384565   0.124349
 -0.0362412  0.084648
  0.0883398  0.0856895
  0.169986   0.100858
  0.0793413  0.0859396
  0.114779   0.0563473
  0.189468   0.0865504
  0.268172   0.109997
  ⋮          
  0.0718065  0.0827917
  0.168601   0.103735
  0.131264   0.0788732
  0.139286   0.124723
  0.205699   0.20088
  0.278883   0.197581
  0.244214   0.236575
  0.21403    0.192462
  0.229819   0.225787

In [131]:
Y = collect(test_returns[2:end,:])
T = n_samples
p = 1
X = hcat(ones(T-1), test_returns[p:end-1,:])
k= 2 #n_regimes
n_variables = 2

Β = [randn(n_variables, n_variables+1) for _ in 1:k]

#= Σ1 = cor2cov([1 .1; .1 1], [.05, .02])
Σ2 = cor2cov([1 .5; .5 1], [.10, .05]) =#

Σ = rand(InverseWishart(4, [.001 0; 0 .015]),k)

init_regimes = rand(k) 
init_regimes = init_regimes ./ sum(init_regimes)

t_m = [.9 .1;.2 .8]

result = expectation_maximisation(Y, X, k, Β, Σ, t_m, init_regimes, 100)



MSVARResult([2.2747923786562207e-159 1.0; 1.0 0.0; … ; 5.967940695096211e-218 1.0; 1.3287843712128291e-216 1.0], [1.023656570395299e-158 0.9999999999999997; 0.9999999999999997 0.0; … ; 7.459925868870263e-219 1.0; 1.3287843712128291e-216 1.0], [NaN NaN; NaN NaN], Any[[-0.2203403215760276 0.026512946262805256; 0.48118264403916383 0.2785479983828855; -0.22194736442445095 0.3806761552758999], [0.08223560400172386 0.030578222902820768; 0.3190606767810442 0.11258170713551423; 0.19605500546597343 0.5891421398639834]], Any[[0.006593246063912858 0.0006690907666883721; 0.0006690907666883721 0.0008666608714666394], [0.007732562520318458 0.0006597474554473553; 0.0006597474554473553 0.0010145962409277049]])

In [132]:
result.smoothed_regimes

999×2 Matrix{Float64}:
 1.02366e-158  1.0
 1.0           0.0
 0.0           1.0
 3.17895e-245  1.0
 2.36969e-226  1.0
 5.53693e-125  1.0
 5.89393e-199  1.0
 1.87765e-205  1.0
 1.47846e-207  1.0
 2.61856e-179  1.0
 ⋮             
 6.93702e-165  1.0
 2.39429e-234  1.0
 5.52327e-164  1.0
 8.30408e-172  1.0
 1.04534e-217  1.0
 1.37717e-263  1.0
 3.72804e-184  1.0
 7.45993e-219  1.0
 1.32878e-216  1.0

In [134]:
result.transition_matrix

2×2 Matrix{Float64}:
 NaN  NaN
 NaN  NaN

In [135]:
initial_regimes_probs(result.transition_matrix)

2-element Vector{Float64}:
 NaN
 NaN